In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

In [2]:
tracks = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
track_elems = pd.read_pickle('../Pickle_Files/track_elems.pkl')

In [3]:
track_audio_features = pd.merge(audio_features, tracks, on='track_uri')
track_audio_features = track_audio_features.drop(['sp_id','id','pos','album_uri','album_name',
                                                  'time_signature','track_name'],axis=1)
track_audio_features['key'] = track_audio_features['key'].astype(int)
track_audio_features['mode'] = track_audio_features['mode'].astype(int)

In [4]:
tracks = pd.merge(track_audio_features, track_elems, on='track_uri')

In [5]:
tracks = tracks[['pid','track_uri','acousticness','danceability','energy','instrumentalness','key','liveness','loudness',
                 'speechiness','tempo','valence']]

In [6]:
d = {'acousticness': 'mean', 'danceability': 'mean','energy': 'mean','instrumentalness':'mean'
     ,'liveness':'mean','loudness':'mean','speechiness':'mean','tempo':'mean','valence':'mean','key':'mean'}

In [7]:
average_playlist = tracks.groupby('pid', as_index=False).aggregate(d).reindex(columns=tracks.columns)

In [8]:
average_playlist = average_playlist.drop(['track_uri'],axis=1)

In [9]:
average_playlist.sample(5)

,pid,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
987592,987592,0.113762,0.741455,0.743568,0.057988,6.613636,0.194643,-5.814000,0.123998,118.096136,0.614057
478393,478393,0.269437,0.632178,0.580556,0.024316,4.844444,0.216338,-7.423844,0.092858,121.000422,0.377484
720670,720670,0.183192,0.583361,0.727021,0.002076,5.298611,0.181067,-6.069521,0.044122,124.331083,0.584486
224302,224302,0.118822,0.712533,0.662156,0.001092,4.800000,0.212211,-7.017467,0.232473,120.590844,0.492500
33078,33078,0.172471,0.667048,0.698119,0.028915,5.523810,0.203364,-8.909071,0.046276,121.815881,0.726429


### Playlists With The Highest Tempo

In [10]:
average_playlist.sort_values(by='tempo', ascending=False).head(2)

,pid,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
67052,67052,0.030518,0.515393,0.661064,0.000221,3.092857,0.117564,-6.793071,0.045785,179.187493,0.46160
986942,986942,0.035484,0.442900,0.895500,0.207404,5.700000,0.302660,-4.405300,0.085930,174.917800,0.30326


#### PID 67052 - Fireflies
<br> This playlist could be said to be made as a joke.
<br> It contains 140 of the exact same tracks, Fireflies - Owl City
<br> This song has been known to be a <i> meme </i> hence its creation more than likley

In [11]:
len(tracks[tracks['pid'] == 67052])

140

In [12]:
tracks[tracks['pid'] == 67052].sample(5)

,pid,track_uri,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
21170886,67052,spotify:track:1mr3616BzLdhXfJmLmRsO8,0.0274,0.513,0.661,0.0,3,0.118,-6.8,0.0439,180.118,0.461
21170873,67052,spotify:track:1mr3616BzLdhXfJmLmRsO8,0.0274,0.513,0.661,0.0,3,0.118,-6.8,0.0439,180.118,0.461
21170818,67052,spotify:track:1mr3616BzLdhXfJmLmRsO8,0.0274,0.513,0.661,0.0,3,0.118,-6.8,0.0439,180.118,0.461
21170854,67052,spotify:track:1mr3616BzLdhXfJmLmRsO8,0.0274,0.513,0.661,0.0,3,0.118,-6.8,0.0439,180.118,0.461
21170905,67052,spotify:track:1mr3616BzLdhXfJmLmRsO8,0.0274,0.513,0.661,0.0,3,0.118,-6.8,0.0439,180.118,0.461


#### PID 986942 - Drum & Bass
<br> This playlist is unlike the previous playlist.
<br> It contains 10 Drum and Bass songs which are all upbeat DJ songs

In [13]:
tracks[tracks['pid'] == 986942].sample(5)

,pid,track_uri,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
57018398,986942,spotify:track:28dEwbbopbWXaXb6cKJuEr,0.002920,0.480,0.942,0.000232,5,0.5250,-2.904,0.3890,175.031,0.226
47039142,986942,spotify:track:6vlQYEyyuT9aF4Bc0GJFeR,0.000441,0.478,0.927,0.070100,8,0.4940,-2.709,0.0980,173.943,0.445
62609562,986942,spotify:track:00BgcUYkIImsDN5rNjkR1U,0.006210,0.362,0.941,0.552000,1,0.0872,-2.001,0.0396,174.953,0.103
42595040,986942,spotify:track:6U7VUPf8d3K1dY0UNMdDXS,0.000479,0.336,0.810,0.024700,11,0.1820,-4.922,0.0338,175.022,0.224
44746164,986942,spotify:track:5wsO7l0ktw11THSZMINzeL,0.007250,0.403,0.946,0.001270,8,0.3000,-3.738,0.0530,176.157,0.482


### Top 5 Playlists with highest Instrumentalness
<br> All these Playlists contain songs with white noise relaxation etc.

In [14]:
average_playlist.sort_values(by='instrumentalness', ascending=False).head(2)

,pid,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
171364,171364,0.799833,0.167783,0.124883,0.955000,5.166667,0.124917,-23.656667,0.042450,108.414500,0.060150
21021,21021,0.946429,0.195286,0.040971,0.949857,6.142857,0.117600,-30.493571,0.045186,111.451429,0.067243


#### PID 171364
<br> This playlist contains 6 songs which are mainly based on pure instrumental features

In [15]:
len(tracks[tracks['pid'] == 171364])

6

In [16]:
tracks[tracks['pid'] == 171364].sample(5)

,pid,track_uri,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
43826118,171364,spotify:track:1fq4mfwRswXZT23ZcomXzl,0.192,0.2000,0.2030,0.884,2,0.1800,-14.981,0.0393,67.428,0.0872
49486598,171364,spotify:track:1OTQ9DbYDRAIQir3Xb3TZr,0.897,0.2050,0.0396,0.986,7,0.1410,-29.014,0.0392,76.308,0.0629
64962050,171364,spotify:track:5A6kqkoGudqIJUkNahv0aX,0.733,0.0837,0.2800,0.953,0,0.0885,-17.697,0.0398,63.797,0.0559
28354653,171364,spotify:track:6cikNK29TJAKKZP8yf7ekP,0.991,0.1340,0.1070,0.967,3,0.1230,-22.308,0.0374,143.842,0.0324
48894687,171364,spotify:track:07A2ZSsP9vyCyyRmo7CikK,0.992,0.1880,0.1130,0.968,9,0.0970,-25.261,0.0448,119.455,0.0388


### Top Playlists with highest Loudness
<br> All these Playlists contain very loud rap and dj songs

In [17]:
average_playlist.sort_values(by='loudness', ascending=False).head(2)

,pid,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
703669,703669,0.113676,0.503667,0.914157,0.482554,4.215686,0.317553,-1.395431,0.164618,127.315980,0.338425
42598,42598,0.022026,0.679920,0.915141,0.331826,5.316583,0.238437,-1.425945,0.215980,162.723854,0.348069


#### PID 703669 - Loud
<br> This playlist very loud drum and bass and instrumental music, we can see that the acousticness of songs contained are very low

In [18]:
len(tracks[tracks['pid'] == 703669])

51

In [19]:
tracks[tracks['pid'] == 703669].sample(5)

,pid,track_uri,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
65379078,703669,spotify:track:4wGwPRhlBieS8okZNrGKxR,0.000301,0.416,0.891,0.55900,1,0.2030,-3.976,0.0332,140.904,0.113
65763144,703669,spotify:track:0ETHZEOubaojKIkG6DdY9c,0.021600,0.404,0.934,0.00151,7,0.4070,0.365,0.2100,175.047,0.541
64265084,703669,spotify:track:6XZ7wxeuezFfYRi2IkBDnv,0.048200,0.764,0.983,0.39000,0,0.2870,1.972,0.3030,146.963,0.595
64428643,703669,spotify:track:0ewUNxLxqFFeXIpYNrIVq6,0.015400,0.513,0.980,0.81300,4,0.6750,0.407,0.1710,154.396,0.430
58145558,703669,spotify:track:7wIJIrxH0BD6vCy0FKsvaw,0.142000,0.810,0.908,0.95300,8,0.0212,0.657,0.1540,107.272,0.811
